In [3]:
from bs4 import BeautifulSoup

import requests

#first loop open the big page with url
#second loop find all links to a place 

url = "www.yelp.co.uk/search?find_loc=praha&cflt=restaurants"

r  = requests.get("http://" +url)

data = r.text

soup = BeautifulSoup(data,"lxml")

place_links = [] #store all sublinks of places

#find all links to places
for link in soup.find_all('a','biz-name'):
    place_links.append(link['href'])


place_pages = []


for place_link in place_links:
    tmpUrl = 'www.yelp.co.uk' + place_link    
    r  = requests.get("http://" +tmpUrl) #getting subpage
    data = r.text
    place_soup = BeautifulSoup(data,"lxml")
    place_pages.append(place_soup)
    
    

In [27]:
dataStorage = []

{'name':'',#done
 'address': '', #done
 'priceRange': '', #done
 'gps': [1,2], #done
 'priceGroup': '', #done
 'stars': '', #done
 'numOfReviews': 0, #done
 'web': '', #done
 'numOfPhotos': '',#done
 'tags': [], #done
 'extras': {}, #done 
 'openHours': { #done
        'Mon' : ['9:00','22:00'],
        'Tue' : ['Closed'],
        'Wed' : '',
        'Thu' : '',
        'Fri' : '',
        'Sat' : '',
        'Sun' : '',
    }
}

i = 1

name = place_pages[i].find('h1','biz-page-title').children.next().encode('utf-8') #need to encode utf-8 for saving into file

#address
address = place_pages[i].find('address').findChildren('span')
street = address[0].text
postCode = address[1].text
city = address[2].text

#price
priceRange = place_pages[i].find('dd','price-description')
priceText = priceRange.text.strip() #clean whitespaces
currency = priceText[:3]
priceText_split = priceText[3:].split('-') 
priceFrom = priceText_split[0]
priceTo = priceText_split[1]

priceGroup = place_pages[i].find('span','price-range')

#location
coordTmp = place_pages[i].find('a','biz-map-directions').findChildren()[0].attrs['src']
coordStart = coordTmp.find('center')
coordTmpShort = coordTmp[coordStart+7:coordStart+28]
gps = coordTmpShort.split('%2C')
gps = map(float,gps) #convert to int

#stars
stars = float(place_pages[i].find('div','rating-very-large').attrs['title'][0:3])

#reviews
numOfReviews = int(place_pages[i].find('span','review-count rating-qualifier').text.strip().split(" ")[0])

#web
webTmp = place_pages[i].find('span','biz-website')

if not webTmp:
    web = ""
else:
    web = webTmp.findChildren()[1].text

#photos
numOfPhotos = place_pages[i].find('a','see-more show-all-overlay')
                  
if numOfPhotos:
    numOfPhotos =  int(numOfPhotos.text.strip().split(" ")[2])
else: 
    numOfPhotos = 0
            
#tags
tags = []

tagsTmp = place_pages[i].find('span','category-str-list').findChildren()

for item in tagsTmp:
    tags.append(item.text)

#extra info, like wifi, takeaway, reservation etc..
extrasTmp = place_pages[i].find('div','short-def-list').findChildren('dl')
extras = {}

for item in extrasTmp:
    itemChildren = item.findChildren()    
    key = itemChildren[0].text.strip()
    val = itemChildren[1].text.strip()
    extras[key] = val
  

openHoursTpl = place_pages[i].find('table','hours-table').find_all('tr')

openHours = {}

for item in openHoursTpl:
    openHourRow = item.findChildren()
    
    dayAbbrKey = openHourRow[0].text
    openHourRange = openHourRow[1].text.strip().split('-')
    
    if(len(openHourRange) == 1):
        openHours[dayAbbrKey] = 'Closed'
    else:
        openFr = openHourRange[0]
        openTo = openHourRange[1]
        openHours[dayAbbrKey] = [openFr,openTo]

{u'Accepts Credit Cards': u'Yes',
 u'Alcohol': u'Beer & Wine Only',
 u'Ambience': u'Casual',
 u'Attire': u'Casual',
 u'Caters': u'Yes',
 u'Delivery': u'No',
 u'Good For': u'Lunch',
 u'Good for Children': u'Yes',
 u'Good for Groups': u'No',
 u'Has TV': u'No',
 u'Noise Level': u'Average',
 u'Outdoor Seating': u'No',
 u'Parking': u'Street',
 u'Take Away': u'Yes',
 u'Takes Reservations': u'No',
 u'Waiter Service': u'No',
 u'Wi-Fi': u'Free'}

In [25]:
#saving into file
with open("Output.txt", "w") as text_file:
    text_file.write("Purchase Amount: {}".format(street))

UnicodeEncodeError: 'ascii' codec can't encode character u'\xe1' in position 5: ordinal not in range(128)